# Classification using TidyModels (Please Rename the File Name before Submission - Must be the same file name)

In this lab we would be going through:
- Logistic Regression
- Linear Discriminant Analysis
- Quadratic Discriminant Analysis

using TidyModels. 

For this lab, we would examining the `OJ` data set that contains a number of numeric variables plus a variable called `Purchase` which has the two labels `CH` and `MM` (which is Citrus Hill or Minute Maid Orange Juice)

In [1]:
suppressPackageStartupMessages(library(tidymodels))
suppressPackageStartupMessages(library(ISLR))
suppressPackageStartupMessages(library(discrim))
suppressPackageStartupMessages(library(corrr))


In [2]:
head(OJ)

,Purchase,WeekofPurchase,StoreID,PriceCH,PriceMM,DiscCH,DiscMM,SpecialCH,SpecialMM,LoyalCH,SalePriceMM,SalePriceCH,PriceDiff,Store7,PctDiscMM,PctDiscCH,ListPriceDiff,STORE
,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>
1,CH,237,1,1.75,1.99,0.00,0.0,0,0,0.500000,1.99,1.75,0.24,No,0.000000,0.000000,0.24,1
2,CH,239,1,1.75,1.99,0.00,0.3,0,1,0.600000,1.69,1.75,-0.06,No,0.150754,0.000000,0.24,1
3,CH,245,1,1.86,2.09,0.17,0.0,0,0,0.680000,2.09,1.69,0.40,No,0.000000,0.091398,0.23,1
4,MM,227,1,1.69,1.69,0.00,0.0,0,0,0.400000,1.69,1.69,0.00,No,0.000000,0.000000,0.00,1
5,CH,228,7,1.69,1.69,0.00,0.0,0,0,0.956535,1.69,1.69,0.00,Yes,0.000000,0.000000,0.00,0
6,CH,230,7,1.69,1.99,0.00,0.0,0,1,0.965228,1.99,1.69,0.30,Yes,0.000000,0.000000,0.30,0


In [3]:
attach(OJ)

In [4]:
str(OJ)

'data.frame':	1070 obs. of  18 variables:
 $ Purchase      : Factor w/ 2 levels "CH","MM": 1 1 1 2 1 1 1 1 1 1 ...
 $ WeekofPurchase: num  237 239 245 227 228 230 232 234 235 238 ...
 $ StoreID       : num  1 1 1 1 7 7 7 7 7 7 ...
 $ PriceCH       : num  1.75 1.75 1.86 1.69 1.69 1.69 1.69 1.75 1.75 1.75 ...
 $ PriceMM       : num  1.99 1.99 2.09 1.69 1.69 1.99 1.99 1.99 1.99 1.99 ...
 $ DiscCH        : num  0 0 0.17 0 0 0 0 0 0 0 ...
 $ DiscMM        : num  0 0.3 0 0 0 0 0.4 0.4 0.4 0.4 ...
 $ SpecialCH     : num  0 0 0 0 0 0 1 1 0 0 ...
 $ SpecialMM     : num  0 1 0 0 0 1 1 0 0 0 ...
 $ LoyalCH       : num  0.5 0.6 0.68 0.4 0.957 ...
 $ SalePriceMM   : num  1.99 1.69 2.09 1.69 1.69 1.99 1.59 1.59 1.59 1.59 ...
 $ SalePriceCH   : num  1.75 1.75 1.69 1.69 1.69 1.69 1.69 1.75 1.75 1.75 ...
 $ PriceDiff     : num  0.24 -0.06 0.4 0 0 0.3 -0.1 -0.16 -0.16 -0.16 ...
 $ Store7        : Factor w/ 2 levels "No","Yes": 1 1 1 1 2 2 2 2 2 2 ...
 $ PctDiscMM     : num  0 0.151 0 0 0 ...
 $ PctDiscC

The `correlate()` function (from `corrr` package) will calculate the correlation matrix between all the variables that it is being fed.

In [ ]:
cor_oj <- OJ %>%
  select(-Purchase, -Store7) %>% #Remove Purchase & Store as it not numeric
  correlate()

Lets pass this correlation to `rplot()` to visualize the correlation matrix

In [ ]:
rplot(cor_oj, colours = c("indianred2", "black", "skyblue1"))

## Logistic Regression

Now we will fit a logistic regression model. We will again use the `parsnip` package, and we will use `logistic_reg()` to create a logistic regression model specification.

In [ ]:
lr_spec <- logistic_reg() %>%
  set_engine("glm") %>% #default engine
  set_mode("classification") #default mode

We want to model the `Direction` of the stock market based on the percentage return from the 5 previous days plus the volume of shares traded. 

In [ ]:
lr_fit <- lr_spec %>%
  fit(
    Purchase ~ PriceCH + PriceMM + SalePriceMM + SalePriceCH + WeekofPurchase,
    data = OJ
    )

lr_fit

In [ ]:
lr_fit %>%
  pluck("fit") %>%
  summary()

The `summary()` lets us see a couple of different things such as; parameter estimates, standard errors, p-values, and model fit statistics. 

we can use the `tidy()` function to extract some of these model attributes for further analysis or presentation.

In [ ]:
tidy(lr_fit)

In [ ]:
predict(lr_fit, new_data = OJ)

The result is a tibble with a single column `.pred_class` which will be a factor variable of the same labels as the original training data set.

We can also get back probability predictions, by specifying `type = "prob"`

In [ ]:
predict(lr_fit, new_data = OJ, type = "prob")

We can describe a `confusion matrix` that would help us understand how well the predictive model is preforming by given a table of predicted values against the true value

`augment()` function helps add the predictions to the `data.frame` and then use that to look at model performance metrics.

In [ ]:
augment(lr_fit, new_data = OJ) %>%
  conf_mat(truth = Purchase, estimate = .pred_class)

 We can represent this as a `heatmap`

In [ ]:
augment(lr_fit, new_data = OJ) %>%
  conf_mat(truth = Purchase, estimate = .pred_class) %>%
  autoplot(type = "heatmap")

A good performing model would ideally have high numbers along the diagonal (up-left to down-right) with small numbers on the off-diagonal. We see here that the model isn’t great, as it tends to predict `"CH"` as `"MM"` more often than it should.

We can also calculate various performance metrics. One of the most common metrics is accuracy, which is how often the model predicted correctly as a percentage.

In [ ]:
augment(lr_fit, new_data = OJ) %>%
  accuracy(truth = Purchase, estimate = .pred_class)

Fitting a model and evaluating the model on the same data would give much information abou the model's performance.

Let us instead split up the data, train it on some of it and then evaluate it on the other part of the data. Since we are working with some data that has a time component,lets train the data over a before a specific week and test it over the set of other weeks.

This would more closely match how such a model would be used in real life.

In [5]:
mean(WeekofPurchase)

[1] 254.3813

In [6]:
oj_train <- OJ %>%
  filter(WeekofPurchase < 260)
dim(oj_train)

oj_test <- OJ %>%
  filter(WeekofPurchase >= 260)
dim(oj_test)

dim(OJ)

[1] 600  18

[1] 470  18

[1] 1070   18

In [7]:
# Build an lr model that fits Purchase as response with other numeric variables
# Predictors: PriceCH, PriceMM, DiscCH, DiscMM, PctDiscMM, PctDiscCH
# Modeled over the training data set created above
lr.fit = function(){
    # Define the logistic regression model
    lr_model <- logistic_reg() %>%
    set_engine("glm") %>%
    set_mode("classification")
    
    
    lr_model %>%
    fit(
      Purchase ~ PriceCH + PriceMM + DiscCH + DiscMM + PctDiscMM + PctDiscCH,
      data = oj_train
    )
    
    
    
}


In [8]:
summary = lr.fit() %>% pluck('fit') %>% summary()
coeff = coef(summary)

stopifnot(round(coeff[1],2) == 1.47) #Intercept test case
stopifnot(round(coeff[2],2) == 2.99) #PriceCH test case

In [12]:
print(coeff[1])
print(coeff[2])

[1] 1.469351
[1] 2.986514


In [13]:
# hidden test cases

In [14]:
# Train the model
lr_model <- lr.fit()

In [15]:
# Use the model to make predictions on the test data
oj_test_predictions <- predict(lr_model, new_data = oj_test)

In [16]:
# Combine predictions with actual Purchase values for evaluation
results <- bind_cols(oj_test_predictions, oj_test %>% select(Purchase))

In [ ]:
results

In [ ]:
# Evaluate the model
confusion_matrix <- results %>%
  conf_mat(truth = Purchase, estimate = .pred_class)

In [ ]:
accuracy <- results %>%
  accuracy(truth = Purchase, estimate = .pred_class)

In [ ]:
# Print the confusion matrix and accuracy
print(confusion_matrix)
print(accuracy)

In [ ]:
### ----------------------------------------------

In [17]:
# Return a confusion matrix and accuracy of the model lr.fit 
# the matrix has to be defined over the test data set 
confusion_matrix = function(){
    confusion_matrix <- results %>%
    conf_mat(truth = Purchase, estimate = .pred_class)
    
}

accuracy.fit = function(){
    accuracy <- results %>%
    accuracy(truth = Purchase, estimate = .pred_class)
    
}

In [18]:
confusion_matrix()


accuracy = accuracy.fit()
stopifnot(round(accuracy[3],2) == 0.70) #Accuracy test case

In [19]:
# hidden test cases

In [21]:
print(confusion_matrix())

          Truth
Prediction  CH  MM
        CH 249  84
        MM  59  78


In [22]:
print(accuracy)

# A tibble: 1 × 3
  .metric  .estimator .estimate
  <chr>    <chr>          <dbl>
1 accuracy binary         0.696


## Linear Discriminant Analysis

We will use the `discrim_linear()` function to create a LDA specification. We are gonna use two predictors (`PriceCH` & `PriceMM`) for easy comparision

In [23]:
lda_spec <- discrim_linear() %>%
  set_mode("classification") %>%
  set_engine("MASS")

In [24]:
lda_fit = lda_spec %>%
  fit(Purchase ~ PriceCH + PriceMM, data = oj_train)

lda_fit

parsnip model object

Fit time:  8ms 
Call:
lda(Purchase ~ PriceCH + PriceMM, data = data)

Prior probabilities of groups:
   CH    MM 
0.575 0.425 

Group means:
    PriceCH  PriceMM
CH 1.831333 2.081101
MM 1.816353 2.022980

Coefficients of linear discriminants:
              LD1
PriceCH  7.069496
PriceMM -8.507458

In [25]:
predict(lda_fit, new_data = oj_test)

.pred_class
<fct>
CH
CH
CH
MM
MM
CH
CH
CH
CH


In [26]:
#confusion matrix
augment(lda_fit, new_data = oj_test) %>%
  conf_mat(truth = Purchase, estimate = .pred_class)

#accuracy 
augment(lda_fit, new_data = oj_test) %>%
      accuracy(truth = Purchase, estimate = .pred_class)

          Truth
Prediction  CH  MM
        CH 219 101
        MM  89  61

.metric,.estimator,.estimate
<chr>,<chr>,<dbl>
accuracy,binary,0.5957447


Lets compare this to `lr()` fit

In [27]:
lda_fit_2 = lda_spec %>%
  fit(Purchase ~ PriceCH + PriceMM + DiscCH + DiscMM + PctDiscMM + PctDiscCH,
      data = oj_train)

#accuracy
augment(lda_fit_2, new_data = oj_test) %>%
      accuracy(truth = Purchase, estimate = .pred_class)

.metric,.estimator,.estimate
<chr>,<chr>,<dbl>
accuracy,binary,0.6957447


## Quadratic Discriminant Analysis

We can fit a `QDA` model by using the `discrim_quad()` function. 

In [28]:
qda_spec = discrim_quad() %>%
  set_mode("classification") %>%
  set_engine("MASS")

`qda_spec` has a similar usage as `lda_spec`. so, 

In [29]:
qda_fit = qda_spec %>% fit(Purchase ~ PriceCH + PriceMM, 
                           data = oj_train)
qda_fit

parsnip model object

Fit time:  5ms 
Call:
qda(Purchase ~ PriceCH + PriceMM, data = data)

Prior probabilities of groups:
   CH    MM 
0.575 0.425 

Group means:
    PriceCH  PriceMM
CH 1.831333 2.081101
MM 1.816353 2.022980

In [30]:
#confusion matrix
augment(qda_fit, new_data = oj_test) %>%
  conf_mat(truth = Purchase, estimate = .pred_class) 

#accuracy
augment(qda_fit, new_data = oj_test) %>%
  accuracy(truth = Purchase, estimate = .pred_class) 

          Truth
Prediction  CH  MM
        CH 308 162
        MM   0   0

.metric,.estimator,.estimate
<chr>,<chr>,<dbl>
accuracy,binary,0.6553191


We can see that, `QDA` performs better compared to `LDA` using two predictors

Now lets compare all the three fits with 6 predictors

In [31]:
qda_fit_2 = qda_spec %>%
  fit(Purchase ~ PriceCH + PriceMM + DiscCH + DiscMM + PctDiscMM + PctDiscCH,
      data = oj_train)

#accuracy
augment(qda_fit_2, new_data = oj_test) %>%
      accuracy(truth = Purchase, estimate = .pred_class)

.metric,.estimator,.estimate
<chr>,<chr>,<dbl>
accuracy,binary,0.7361702


In [32]:
get_accuracy = function(fit){
    accuracy = augment(fit, new_data = oj_test) %>% 
      accuracy(truth = Purchase, estimate = .pred_class)
    accuracy[3]
}
accuracy_matrix = matrix(c( get_accuracy(lr.fit())
                           , get_accuracy(lda_fit_2)
                           , get_accuracy(qda_fit_2))
                           , nrow = 1, ncol = 3, byrow=FALSE)
colnames(accuracy_matrix) = c("LR", "LDA", "QDA")
accuracy_matrix

LR,LDA,QDA
<dbl>,<dbl>,<dbl>
0.6957447,0.6957447,0.7361702


we can see that the `QDA` works better for this data where as `LDA` and `LR` work similary for this data set